<h1>Исследование данных по csv</h1>
Попробуем найти требуемые компании в таблице <code>yc_csv.csv</code>

In [17]:
import pandas as pd
df = pd.read_csv("assets/yc_csv.csv")

In [18]:
df

,xid,#,Companies,Company ID,Deal ID,Deal Date,Announced Date,Deal Size,Pre-money Valuation,Post Valuation,...,Lead/Sole Investors,Employees,Revenue Growth since last debt deal,Revenue,EBITDA,Total Debt (from financials),Deal Synopsis,Financing Status Note,CEO (at time of deal),CEO PBId
0,48635,48386,Parakey (Software Development Applications),52732-99,19117-54T,17/11/2006 00:00,NaN,1.56,13.27,14.83,...,NaN,NaN,NaN,NaN,NaN,NaN,The company raised $1.56 million of Series A f...,The company was acquired by Facebook (NASDAQ: ...,{'62492-86P': 'Blake Ross'},62492-86P
1,58836,58587,Snaptalent,61729-75,32010-85T,01/01/2008 00:00,NaN,2.00,4.03,6.03,...,NaN,NaN,NaN,NaN,NaN,NaN,The company joined Y Combinator as part of the...,The company is no longer actively in business ...,NaN,NaN
2,60141,59892,FriendFeed,41728-33,19653-94T,25/02/2008 00:00,NaN,5.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The company raised $5 million of Series A vent...,The company was acquired by Facebook for $50 m...,{'167058-55P': 'Jim Norris'},167058-55P
3,64785,64536,RescueTime,52794-46,135800-20T,23/09/2008 00:00,NaN,0.90,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"The company raised $900,000 of seed funding fr...","True Ventures, Y Combinator, Lowercase Capital...",{'43621-03P': 'Brian Fioca'},43621-03P
4,66671,66422,CarWoo,51122-80,18676-81T,01/01/2009 00:00,NaN,1.93,2.69,4.62,...,NaN,NaN,NaN,NaN,NaN,NaN,The company raised $1.9 million of seed fundin...,The company is no longer actively in business ...,{'38143-27P': 'Robert McClung'},38143-27P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,314141,313892,Reflex (Software Development Applications),520530-40,223029-82T,02/08/2023 00:00,NaN,5.00,NaN,NaN,...,{'11229-04': 'Lux Capital'},NaN,NaN,NaN,NaN,NaN,The company raised $5 million of seed funding ...,The company raised $5 million of seed funding ...,{'336293-83P': 'Nikhil Rao'},336293-83P
1343,314262,314013,MindsDB,223449-22,212197-42T,08/08/2023 00:00,07/02/2023 00:00,46.50,NaN,NaN,...,{'11133-01': 'Benchmark (San Francisco) (Cheta...,NaN,NaN,NaN,NaN,NaN,The company raised $46.5 million of Series A v...,The company raised $46.5 million of Series A v...,{'100780-12P': 'Jorge Torres'},100780-12P
1344,314273,314024,Daybreak Health,434212-21,233351-02T,08/08/2023 00:00,NaN,13.00,52.00,65.00,...,{'11323-45': 'Union Square Ventures'},NaN,NaN,NaN,NaN,NaN,The company raised $13 million of Series B ven...,The company raised $13 million of Series B ven...,{'227761-57P': 'Alex Alvarado'},227761-57P
1345,314428,314179,Arpio,435028-33,219305-98T,14/08/2023 00:00,28/07/2023 00:00,8.20,NaN,NaN,...,"{'342995-41': 'Companyon Ventures', '42943-96'...",NaN,NaN,NaN,NaN,NaN,The company raised $7.98 million of seed fundi...,The company raised $7.98 million of seed fundi...,{'229843-99P': 'Douglas Neumann'},229843-99P


In [19]:
investors = df["Investors"]
investors

0       {'42715-09': 'Baseline Ventures', '10521-55': ...
1       {'51007-51': 'Betaworks', '42715-00': 'Felicis...
2       {'11133-01': 'Benchmark (San Francisco)', '404...
3       {'106007-68': 'Christopher Sacca (Christopher ...
4       {'50987-89': 'Accelerator Ventures', '11127-25...
                              ...                        
1342    {'179440-93': 'Abstract Ventures', '11229-04':...
1343    {'11133-01': 'Benchmark (San Francisco) (Cheta...
1344    {'11333-26': 'Lightspeed Venture Partners', '1...
1345    {'342995-41': 'Companyon Ventures', '266227-21...
1346    {'164406-34': 'Afore Capital', '432967-42': 'C...
Name: Investors, Length: 1347, dtype: object

In [20]:
companies = df["Companies"]
companies

0       Parakey (Software Development Applications)
1                                        Snaptalent
2                                        FriendFeed
3                                        RescueTime
4                                            CarWoo
                           ...                     
1342     Reflex (Software Development Applications)
1343                                        MindsDB
1344                                Daybreak Health
1345                                          Arpio
1346                                   highlight.io
Name: Companies, Length: 1347, dtype: object

In [21]:
companies_to_investigate = ["OpenPipe", "CodeStory", "Flair Health", "Olio Labs", "Andromeda Surgical", "Quill AI", "Martin", "Orbio Earth", "Taylor AI", "Remy Security", "Cardinal Gray", "truemetrics", "FlowiseAI", "Nowadays", "Converge", "Cargo", "Flex", "Kino AI"]
len(companies_to_investigate)

18

In [22]:
found_companies = 0
for idx in range(len(companies)):
    company = companies[idx]
    if company in companies_to_investigate:
        found_companies += 1
        print(company + ": " + investors[idx])
print("Found " + str(found_companies) + " companies in given dataframe")

Found 0 companies in given dataframe


**Вывод:** к сожалению, не было найдено ни одной компании из собранной базы данных.